In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sea
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import xgboost as xgb
import tensorflow as tf


In [6]:
df = pd.read_csv('SOLUSTDAtas_tratado.csv')
df.head()

,date,open,high,low,close
0,2020-08-12 03:00:00,3.10,3.35,3.10,3.25
1,2020-08-12 04:00:00,3.25,3.25,3.15,3.15
2,2020-08-12 05:00:00,3.15,3.30,3.15,3.30
3,2020-08-12 06:00:00,3.30,3.30,3.15,3.30
4,2020-08-12 07:00:00,3.25,3.25,3.20,3.25


In [7]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36400 entries, 0 to 36399
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    36400 non-null  object 
 1   open    36400 non-null  float64
 2   high    36400 non-null  float64
 3   low     36400 non-null  float64
 4   close   36400 non-null  float64
dtypes: float64(4), object(1)
memory usage: 1.4+ MB
None


# Preprocesado de Datos

In [8]:
tamanio = df.shape[0]

In [9]:
df_train = df.copy().loc[0:int(tamanio*0.7)]
df_train

,date,open,high,low,close
0,2020-08-12 03:00:00,3.10,3.35,3.10,3.25
1,2020-08-12 04:00:00,3.25,3.25,3.15,3.15
2,2020-08-12 05:00:00,3.15,3.30,3.15,3.30
3,2020-08-12 06:00:00,3.30,3.30,3.15,3.30
4,2020-08-12 07:00:00,3.25,3.25,3.20,3.25
...,...,...,...,...,...
25476,2023-07-11 21:00:00,22.00,22.05,21.90,22.00
25477,2023-07-11 22:00:00,22.00,22.10,21.90,22.00
25478,2023-07-11 23:00:00,22.00,22.05,21.75,21.95
25479,2023-07-12 00:00:00,21.95,22.10,21.90,22.05


In [10]:
df_vali = df.copy().loc[int(tamanio*0.7 + 1):int(tamanio*0.9)]
df_vali

,date,open,high,low,close
25481,2023-07-12 02:00:00,22.10,22.30,22.05,22.15
25482,2023-07-12 03:00:00,22.15,22.25,22.10,22.10
25483,2023-07-12 04:00:00,22.10,22.10,22.00,22.00
25484,2023-07-12 05:00:00,22.00,22.00,21.90,21.95
25485,2023-07-12 06:00:00,21.95,22.05,21.90,22.00
...,...,...,...,...,...
32756,2024-05-10 05:00:00,153.65,154.35,152.85,153.95
32757,2024-05-10 06:00:00,153.95,154.70,153.45,153.75
32758,2024-05-10 07:00:00,153.75,154.10,152.30,153.30
32759,2024-05-10 08:00:00,153.30,155.10,153.15,154.95


In [11]:
df_test = df.copy().loc[int(tamanio*0.9 + 1):tamanio]
df_test

,date,open,high,low,close
32761,2024-05-10 10:00:00,154.35,154.5,153.45,154.10
32762,2024-05-10 11:00:00,154.10,154.8,153.25,154.15
32763,2024-05-10 12:00:00,154.15,154.3,153.25,154.15
32764,2024-05-10 13:00:00,154.15,155.2,153.00,155.05
32765,2024-05-10 14:00:00,155.05,155.4,153.10,153.30
...,...,...,...,...,...
36395,2024-10-08 20:00:00,143.35,143.9,142.35,142.95
36396,2024-10-08 21:00:00,142.95,144.1,142.25,143.75
36397,2024-10-08 22:00:00,143.75,144.5,143.35,144.50
36398,2024-10-08 23:00:00,144.50,144.7,144.05,144.25


In [12]:
df_valitest = pd.concat([df_vali, df_test], axis=0)

Numero de horas que se utilizan en la predicción

In [13]:
numhorasconst = 4

# Declaramos y Entrenamos el modelo

Función que prepara los datos

In [14]:
def preparar_datosRandomForest(df, numhoras):
    X = []
    y= []
    for i in range(0, df.shape[0] - numhoras):
        auxy = df.iloc[i + numhoras]
        y.append(auxy.close)
        aux1 = []
        for e in range(0, numhoras):
            aux = df.iloc[i + e]
            for r in range(1, aux.size):
                aux1.append(aux[r])
        X.append(aux1)
    return (X, y)

Preparamos los datos en el formato requerido por el algoritmo para su entrenamiento

In [15]:
data = preparar_datosRandomForest(df_train, numhorasconst)
X1 = data[0]
y1 = data[1]

C:\Users\raulg\AppData\Local\Temp\ipykernel_4808\2914413355.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  aux1.append(aux[r])


Preparamos los datos usados para testear el algortimo en el formato requerido

In [16]:
data = preparar_datosRandomForest(df_valitest, numhorasconst)
Xvt = data[0]
yvt = data[1]

C:\Users\raulg\AppData\Local\Temp\ipykernel_4808\2914413355.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  aux1.append(aux[r])


Desarrollamos un algoritmo para entrenar distintas instacias con distintos parámetros y compararlas

In [17]:
def evalRandomForest(Testrpr, predictT):
    suma = 0
    n = len(Testrpr)
    for i in range(0,n):
        suma = abs(predictT[i] - Testrpr[i])/Testrpr[i] +  suma
    error_medio = suma/n
    emp = error_medio*100 # error medio en porcentaje
    return emp

# Predicción Utilizando XGBoost

In [18]:
def create_df_n(df, n):
    df_aux = df.copy()
    for i in range(1, n):
        df_aux['open_before' + str(i)] = df_aux['open'].shift(+i)
        df_aux['high_before' + str(i)] = df_aux['high'].shift(+i)
        df_aux['low_before' + str(i)] = df_aux['low'].shift(+i)
        df_aux['close_before' + str(i)] = df_aux['close'].shift(+i)
        df_aux['value_before' + str(i)] = df_aux['value'].shift(+i)
    df_aux['close_next'] = df_aux['close'].shift(-1)
    df_aux = df_aux.dropna()
    return df_aux

Ponemos los datos en el formato correcto

In [19]:
df_xgb = create_df_n(df, 1)


In [20]:
tamanio_xgb = df_xgb.shape[0]
tamanio_xgb

36399

In [21]:
def createdftrain(df_aux):
    tamanio_aux = df_aux.shape[0]
    return df_aux.copy().iloc[0:int(tamanio_aux*0.7)]

In [22]:
def createdfvali(df_aux):
    tamanio_aux = df_aux.shape[0]
    return df_aux.copy().iloc[int(tamanio_aux*0.7 + 1):int(tamanio_aux*0.9)]

In [23]:
def createdftest(df_aux):
    tamanio_aux = df_aux.shape[0]
    return df_aux.copy().iloc[int(tamanio_aux*0.9 + 1):tamanio_aux]

In [24]:
def preparar_datosXGBoost(df_aux):
    X = df_aux.drop(['date', 'close_next'], axis=1)
    #X = df_aux[['open', 'high', 'low', 'close', 'open_before1', 'high_before1', 'low_before1', 'close_before1']] 
    y = df_aux['close_next']
    return (xgb.DMatrix(data=X, label=y), y)

In [25]:
df_train_xgb = createdftrain(df_xgb)
df_train_xgb

,date,open,high,low,close,close_next
0,2020-08-12 03:00:00,3.10,3.35,3.10,3.25,3.15
1,2020-08-12 04:00:00,3.25,3.25,3.15,3.15,3.30
2,2020-08-12 05:00:00,3.15,3.30,3.15,3.30,3.30
3,2020-08-12 06:00:00,3.30,3.30,3.15,3.30,3.25
4,2020-08-12 07:00:00,3.25,3.25,3.20,3.25,3.25
...,...,...,...,...,...,...
25474,2023-07-11 19:00:00,22.10,22.20,22.00,22.10,22.00
25475,2023-07-11 20:00:00,22.10,22.10,21.95,22.00,22.00
25476,2023-07-11 21:00:00,22.00,22.05,21.90,22.00,22.00
25477,2023-07-11 22:00:00,22.00,22.10,21.90,22.00,21.95


In [26]:
df_vali_xgb = createdfvali(df_xgb)
df_vali_xgb

,date,open,high,low,close,close_next
25480,2023-07-12 01:00:00,22.05,22.15,22.00,22.10,22.15
25481,2023-07-12 02:00:00,22.10,22.30,22.05,22.15,22.10
25482,2023-07-12 03:00:00,22.15,22.25,22.10,22.10,22.00
25483,2023-07-12 04:00:00,22.10,22.10,22.00,22.00,21.95
25484,2023-07-12 05:00:00,22.00,22.00,21.90,21.95,22.00
...,...,...,...,...,...,...
32754,2024-05-10 03:00:00,152.05,155.00,151.45,153.40,153.65
32755,2024-05-10 04:00:00,153.40,153.80,152.70,153.65,153.95
32756,2024-05-10 05:00:00,153.65,154.35,152.85,153.95,153.75
32757,2024-05-10 06:00:00,153.95,154.70,153.45,153.75,153.30


In [27]:
df_test_xgb = createdftest(df_xgb)
df_test_xgb

,date,open,high,low,close,close_next
32760,2024-05-10 09:00:00,154.95,155.75,154.25,154.35,154.10
32761,2024-05-10 10:00:00,154.35,154.50,153.45,154.10,154.15
32762,2024-05-10 11:00:00,154.10,154.80,153.25,154.15,154.15
32763,2024-05-10 12:00:00,154.15,154.30,153.25,154.15,155.05
32764,2024-05-10 13:00:00,154.15,155.20,153.00,155.05,153.30
...,...,...,...,...,...,...
36394,2024-10-08 19:00:00,143.80,143.95,142.40,143.35,142.95
36395,2024-10-08 20:00:00,143.35,143.90,142.35,142.95,143.75
36396,2024-10-08 21:00:00,142.95,144.10,142.25,143.75,144.50
36397,2024-10-08 22:00:00,143.75,144.50,143.35,144.50,144.25


In [28]:
df_valitest_xgb = pd.concat([df_vali_xgb, df_test_xgb], ignore_index=True)

In [29]:
XvtXGB = df_valitest_xgb[['open', 'high', 'low', 'close', 'value']] 
yvtXGB = df_valitest_xgb['close_next']

KeyError: "['value'] not in index"

In [ ]:
dtrainvali = xgb.DMatrix(data=XvtXGB, label=yvtXGB)

Definimos las características para el entrenamiento

Comprobación de la tasa de error en los datos de test

In [ ]:
def evalXGB(Test_xgb, predict_xgb_test):
    suma = 0
    n = len(Test_xgb)
    for i in range(0,n):
        suma = abs(predict_xgb_test[i] - Test_xgb[i])/Test_xgb[i] +  suma
    error_medio = suma/n
    emp = error_medio*100 # error medio en porcentaje
    return emp

# Redes neuronales Densas

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import load_model

In [31]:
print("¿GPU detectada?:", tf.config.list_physical_devices('GPU'))

¿GPU detectada?: []


In [32]:
print("Versión de TensorFlow:", tf.__version__)

Versión de TensorFlow: 2.17.0


In [34]:


modelo_ruta = "mi_modelo_lstmICM3_78_0.5872.h5"

#modelo = load_model(modelo_ruta)
modelo = load_model("mi_modelo_lstmICM3_78_0.5872.h5", compile=False)

print(modelo.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        17,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,985 (70.25 KB)

 Trainable params: 17,985 (70.25 KB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i+n_steps])
        y.append(data[i+n_steps, 3])  
    return np.array(X), np.array(y)

In [ ]:
def preparar_datos(df, numhoras):
    data = df[['open', 'high', 'low', 'close', 'value']].values
    X, y = create_sequences(data, numhoras)
    X_aux = []
    for i in X:
        aux = []
        for r in range(0, numhoras):
            for elem in i[r]:
                aux.append(elem)
        X_aux.append(aux)       
    X_aux = np.array(X_aux) 
    return X_aux, y

In [ ]:
def evalRedDensa(ytest, y_pred):
    y_pred = y_pred.flatten()
    suma = 0
    n = len(y_pred)
    for i in range(0,n):
        suma = abs(y_pred[i] - ytest[i])/ytest[i] +  suma
    error_medio = suma/n
    emp = error_medio*100 # error medio en porcentaje
    return emp

In [ ]:
def opti_redes_densas(epoch_ini, epoch_fin, batch_array, numhoras, X_train, y_train, X_vali, y_vali, X_test, y_test):
    best = 100
    epoch_best = 0
    bacth_best = 0
    best_model = None
    for e in range(epoch_ini, epoch_fin + 1):
        for b in batch_array:
            model = Sequential()
            model.add(Dense(64, activation='relu', input_shape=(numhoras*5,)))
            model.add(Dense(64, activation='relu'))
            model.add(Dense(1))
            model.compile(optimizer='adam', loss='mape')
            history = model.fit(X_train, y_train, epochs=e, batch_size=b, validation_data=(X_vali, y_vali), shuffle=False)
            y_pred = model.predict(X_test)
            valor = evalRedDensa(y_test, y_pred)
            print("epoch: "+str(e)+", batch_size: "+str(b)+", value: "+str(valor))
            if valor < best:
                best = valor
                epoch_best = e
                bacth_best = b
                cadena_guardado = "ModelosDensosOptiMoreDataIMC/mi_modelo_denso_Opti_e"+str(e)+"_b"+str(b)+"_v"+str(round(valor, 3))
                model.save(cadena_guardado+".h5")
                model.save(cadena_guardado+".keras")
                best_model = model
    return epoch_best, bacth_best, valor, best_model 

In [ ]:
def opti_rd_h(inih, finh, epoch_ini, epoch_fin, batch_array):
    best = 100
    epoch_best = 0
    bacth_best = 0
    h_best = 0
    best_model = None
    for i in range(inih, finh+1):
        Xtrain, ytrain = preparar_datos(df_train, i)
        Xvali, yvali = preparar_datos(df_vali, i)
        Xtest, ytest = preparar_datos(df_test, i)
        valores = opti_redes_densas(epoch_ini, epoch_fin, batch_array, i, Xtrain, ytrain, Xvali, yvali, Xtest, ytest)
        if valores[2] < best:
            best = valores[2]
            epoch_best = valores[0]
            bacth_best = valores[1]
            h_best = i
            best_model = valores[3]
            cadena_guardado = "ModelosDensosOptiMoreDataIMCBest/mi_modelo_denso_Opti_e"+str(epoch_best)+"_b"+str(bacth_best)+"_h"+str(i)+"_v"+str(round(best, 3))
            best_model.save(cadena_guardado+".h5")
            best_model.save(cadena_guardado+".keras")
    return best, epoch_best, bacth_best, h_best, best_model

In [ ]:
data = opti_rd_h(7, 16, 3, 15, [4, 6, 8, 12, 16, 24, 32, 46, 64, 96, 128, 256])

# Redes neuronales LSTM

In [35]:
from tensorflow.keras.layers import LSTM

In [36]:
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i+n_steps])
        y.append(data[i+n_steps, 3])  
    return np.array(X), np.array(y)

In [38]:
def preparar_datosLSTM(df, numhoras):
    return create_sequences(df[['open', 'high', 'low', 'close']].values, numhoras)

In [39]:
X_test, y_test = preparar_datosLSTM(df_test, 7)

In [40]:
pred = modelo.predict(X_test)

ValueError: Exception encountered when calling LSTMCell.call().

[1mDimensions must be equal, but are 4 and 5 for '{{node sequential_1_1/lstm_1/lstm_cell_1/MatMul}} = MatMul[T=DT_FLOAT, grad_a=false, grad_b=false, transpose_a=false, transpose_b=false](sequential_1_1/lstm_1/strided_slice_1, sequential_1_1/lstm_1/lstm_cell_1/Cast/ReadVariableOp)' with input shapes: [32,4], [5,256].[0m

Arguments received by LSTMCell.call():
  • inputs=tf.Tensor(shape=(32, 4), dtype=float32)
  • states=('tf.Tensor(shape=(32, 64), dtype=float32)', 'tf.Tensor(shape=(32, 64), dtype=float32)')
  • training=False

In [ ]:
def evalRedLSTM(ytest, y_pred):
    y_pred = y_pred.flatten()
    suma = 0
    n = len(y_pred)
    for i in range(0,n):
        suma = abs(y_pred[i] - ytest[i])/ytest[i] +  suma
    error_medio = suma/n
    emp = error_medio*100 # error medio en porcentaje
    return emp

In [ ]:
def opti_redes_LSTM(epoch_ini, epoch_fin, batch_array, X_trainLSTM, y_trainLSTM, X_valiLSTM, y_valiLSTM, X_testLSTM, y_testLSTM, numhoras):
    best = 100
    epoch_best = 0
    bacth_best = 0
    best_model = None
    for e in range(epoch_ini, epoch_fin + 1):
        for b in batch_array:
            modelLSTM = Sequential()
            modelLSTM.add(LSTM(64, activation='relu', input_shape=(numhoras, 5)))
            modelLSTM.add(Dense(1))
            modelLSTM.compile(optimizer='adam', loss='mape')
            historyLSTM = modelLSTM.fit(X_trainLSTM, y_trainLSTM, epochs=e, batch_size=b, validation_data=(X_valiLSTM, y_valiLSTM), shuffle=False)
            y_pred = modelLSTM.predict(X_testLSTM)
            valor = evalRedLSTM(y_testLSTM, y_pred)
            print("epoch: "+str(e)+", batch_size: "+str(b)+", value: "+str(valor))
            if valor < best:
                best = valor
                epoch_best = e
                bacth_best = b
                best_model = modelLSTM
                cadena_guardado = "ModelosLSTMOptiMoreDataIMC/mi_modelo_LSTMOpti_e"+str(e)+"_b"+str(b)+"_v"+str(round(valor, 3))
                best_model.save(cadena_guardado+".h5")
                best_model.save(cadena_guardado+".keras")
    return epoch_best, bacth_best, valor, best_model

In [ ]:
BATCH_ARRAY = [4, 8, 12, 16, 20, 24, 28, 32, 40, 48, 64, 96, 128, 192, 256]

In [ ]:
def opti_rLSTM_h(inih, finh, epoch_ini, epoch_fin, batch_array):
    best = 100
    epoch_best = 0
    bacth_best = 0
    h_best = 0
    best_model = None
    for i in range(inih, finh+1):
        Xtrain, ytrain = preparar_datosLSTM(df_train, i)
        Xvali, yvali = preparar_datosLSTM(df_vali, i)
        Xtest, ytest = preparar_datosLSTM(df_test, i)
        valores = opti_redes_LSTM(epoch_ini, epoch_fin, batch_array, Xtrain, ytrain, Xvali, yvali, Xtest, ytest, i)
        if valores[2] < best:
            best = valores[2]
            epoch_best = valores[0]
            bacth_best = valores[1]
            h_best = i
            best_model = valores[3]
            cadena_guardado = "ModelosLSTMOptiMoreDataIMCBest/mi_modelo_LSTM_Opti_e"+str(epoch_best)+"_b"+str(bacth_best)+"_h"+str(i)+"_v"+str(round(best, 3))
            best_model.save(cadena_guardado+".h5")
            best_model.save(cadena_guardado+".keras")
    return best, epoch_best, bacth_best, h_best, best_model

In [ ]:
data = opti_rLSTM_h(7, 16, 3, 20, BATCH_ARRAY)